# Data Analysis

In [10]:
import pandas as pd

df = pd.read_json('responses.json', lines=True)
df = df[['prompt', 'response']]

## Sentiment Analysis with Twitter-roBERTa-base for Sentiment Analysis with and without fine-tuning

In [11]:
import os
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments, pipeline
from datasets import load_dataset
from torch.utils.data import DataLoader


sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    tokenizer="cardiffnlp/twitter-roberta-base-sentiment-latest",
    truncation=True,
    padding=True,
    max_length=512
)

def chunk_text(text, chunk_size=512, overlap=50):
    words = text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
        i += chunk_size - overlap  
    return chunks

def analyze_long_sentiment(text):
    chunks = chunk_text(text)
    results = sentiment_pipeline(chunks)  
    scores = {"positive": 0, "neutral": 0, "negative": 0}  
    for result in results:
        scores[result['label']] += result['score']

    for label in scores:
        scores[label] /= len(scores)  
    
    final_label = max(scores, key=scores.get)
    return {"label": final_label, "score": scores[final_label]}


df['sentiment'] = df['response'].apply(lambda x: analyze_long_sentiment(x))

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


In [ ]:
dataset = load_dataset("classla/parlasent", "EN")
model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3, ignore_mismatched_sizes=True)

path = "./fine_tuned_roberta_parlasent"

if not os.path.exists(path):
    def tokenize_function(examples):
        return tokenizer(examples["sentence"], padding="max_length", truncation=True, max_length=512)

    def encode_labels(example):
        example['label'] = label_mapping[example['label']]
        return example

    tokenized_datasets = dataset.map(tokenize_function, batched=True)
    tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.2)
    label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}

    tokenized_datasets = tokenized_datasets.map(encode_labels)
    tokenized_datasets = tokenized_datasets.remove_columns(["sentence"])
    tokenized_datasets.set_format("torch")

    training_args = TrainingArguments(
        output_dir=path,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
    )

    trainer.train()
    trainer.save_model("./fine_tuned_roberta_parlasent")


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
from torch import softmax

def analyze_sentiment_finetuned(text: str) -> float:
    def get_label(probs):
        labels = ['Negative', 'Neutral', 'Positive']
        return labels[probs.argmax()]
    
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = {key: value.to(model.device) for key, value in inputs.items()}  # Move inputs to the same device as the model
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    
    probs = softmax(logits, dim=-1).squeeze().cpu().numpy()  
    return {'label': get_label(probs), 'score': probs[probs.argmax()]}

df['sentiment_score_finetuned'] = df['response'].apply(lambda x: analyze_sentiment_finetuned(x))

## Entity recognition with BERT-base-ner

In [14]:
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

def mentions_china_pipeline(text):
    entities = ner_pipeline(text)
    for entity in entities:
        if entity["entity_group"] == "LOC" and entity["word"].lower() in {"china", "beijing", "cpc", "chinese communist party"}:
            return True
    return False


df['prompt_about_china'] = df['prompt'].apply(lambda x: mentions_china_pipeline(x))
df['response_about_china'] = df['response'].apply(lambda x: mentions_china_pipeline(x))

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


## Entity recognition with dbmdz/bert-large-cased-finetuned-conll03-english

In [15]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

model_name = "dbmdz/bert-large-cased-finetuned-conll03-english"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)

ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

china_related_keywords = {
    "China", "Chinese", "Beijing", "CPC", "Communist Party of China",
    "Huawei", "Alibaba", "Belt and Road Initiative", "Xi Jinping"
}

def mentions_china_pipeline_finetuned(text):
    """ Esegue Named Entity Recognition (NER) e filtra le entità correlate alla Cina """
    entities = ner_pipeline(text)
    for word in entities:
        if word['word'] in china_related_keywords:
            return True
    return False


df['prompt_about_china_finetuned'] = df['prompt'].apply(lambda x: mentions_china_pipeline_finetuned(x))
df['response_about_china_finetuned'] = df['response'].apply(lambda x: mentions_china_pipeline_finetuned(x))


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


## Zero-shot stance classification with BERT-large-mnli

In [16]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def get_stance_scores(text):
    targets = ["capitalism", "communism"]
    hypothesis_templates = [f"This text is about {target}" for target in targets]
    
    results = classifier(text, hypothesis_templates, multi_label=False)
    
    stance_results = {label: score for label, score in zip(results["labels"], results["scores"])}
    highest_stance = max(stance_results, key=stance_results.get)
    lowest_stance = min(stance_results, key=stance_results.get)
    if stance_results[highest_stance] - stance_results[lowest_stance] < 0.2:
        return {'label' : "neutral", 'score' : 1 - (stance_results[highest_stance] - stance_results[lowest_stance])}
    score = stance_results[highest_stance]
    highest_stance = highest_stance[len("This text is "):]
    return {'label' : highest_stance, 'score' : score}


df['prompt_stance'] = df['prompt'].apply(lambda x: get_stance_scores(x))
df['response_stance'] = df['response'].apply(lambda x: get_stance_scores(x))


Device set to use cuda:0


## Left-Right Wing stance classification with politicalBiasBERT

In [17]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForSequenceClassification.from_pretrained("bucketresearch/politicalBiasBERT")

def get_stance(text):
    def get_label(logits):
        return ["left wing", "center", "right wing"][logits.argmax().item()]

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    labels = torch.tensor([0])
    outputs = model(**inputs, labels=labels)
    loss, logits = outputs[:2]

    label = get_label(logits.softmax(dim=-1)[0])
    score = logits.softmax(dim=-1)[0].max().item()
    return {'label': label, 'score': score}

df['left_right_stance'] = df['response'].apply(lambda x: get_stance(x))

## saving results

In [18]:
if os.path.exists('processed_results.json'):
    if os.path.exists('processed_results_old.json'):
        os.remove('processed_results_old.json')
    os.rename('processed_results.json', 'processed_results_old.json')
df.to_json('processed_results.json', orient='records')